In [2]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from glob import glob

In [32]:
from tobac_flow.dataloader import fill_time_gap_nan

def glob_seviri_files(start_date, end_date, 
                      file_type="secondary", 
                      file_path="../data/SEVIRI_ORAC/"):
    if file_type not in ["secondary", "cloud", "flux"]:
        raise ValueError("file_type parameter must be one of 'secondary', 'cloud' or 'flux'")
    
    dates = pd.date_range(start_date, end_date, freq='H', closed='left').to_pydatetime()
    
    seviri_files = []
    
    for date in dates:
        datestr = date.strftime("%Y%m%d%H")
        if file_type == "secondary":
            glob_str = f"H-000-MSG3__-MSG3________-_________-EPI______-{datestr}*-__.secondary.nc"
        if file_type == "cloud":
            glob_str = f"{datestr}*00-ESACCI-L2_CLOUD-CLD_PRODUCTS-SEVIRI-MSG3-fv1.0.nc"
        if file_type == "flux":
            glob_str = f"{datestr}*00-ESACCI-TOA-SEVIRI-MSG3-fv1.0.nc"
        seviri_files.extend(glob(os.path.join(file_path, glob_str)))
    
    return sorted(seviri_files)

def find_seviri_files(start_date, end_date, n_pad_files=1, 
                      file_type="secondary", 
                      file_path="../data/SEVIRI_ORAC/"):
    
    seviri_files = glob_seviri_files(start_date, end_date, 
                                     file_type, file_path)
    
    if n_pad_files > 0:
        pad_hours = int(np.ceil(n_pad_files/4))

        seviri_pre_file = glob_seviri_files(start_date-timedelta(hours=pad_hours), 
                                            start_date, file_type, file_path)
        if len(seviri_pre_file):
            seviri_pre_file = seviri_pre_file[-n_pad_files:]
        
        seviri_post_file = glob_seviri_files(end_date, 
                                             end_date+timedelta(hours=pad_hours), 
                                             file_type, file_path)
        if len(seviri_post_file):
            seviri_post_file = seviri_post_file[:n_pad_files]

        seviri_files = seviri_pre_file + seviri_files + seviri_post_file
        
    return seviri_files

def load_seviri_dataset(seviri_files, x0=None, x1=None, y0=None, y1=None):
    seviri_ds = xr.open_mfdataset(seviri_files, 
                                  combine="nested", 
                                  concat_dim="t").isel(across_track=slice(x0,x1), along_track=slice(y0,y1))
    
    seviri_ds = seviri_ds.assign_coords(t=[parse_date(f[-28:-16]) for f in seviri_files])
    
    return seviri_ds
    

def seviri_dataloader(start_date, end_date, n_pad_files=1,
                      file_path="../data/SEVIRI_ORAC/", 
                      x0=None, x1=None, y0=None, y1=None, 
                      time_gap=timedelta(minutes=30), 
                      dtype=np.float32, return_new_ds=False):
    
    seviri_files = find_seviri_files(start_date, end_date, 
                                     n_pad_files=n_pad_files,
                                     file_path=file_path)
    
    seviri_ds = load_seviri_dataset(seviri_files, x0, x1, y0, y1)
    
    bt = seviri_ds.brightness_temperature_in_channel_no_9.astype(dtype)
    try:
        bt = bt.compute()
    except AttributeError:
        pass
    
    wvd = (seviri_ds.brightness_temperature_in_channel_no_5 - seviri_ds.brightness_temperature_in_channel_no_6).astype(dtype)
    try:
        wvd = wvd.compute()
    except AttributeError:
        pass

    
    swd = (seviri_ds.brightness_temperature_in_channel_no_9-seviri_ds.brightness_temperature_in_channel_no_10).astype(dtype)
    try:
        swd = swd.compute()
    except AttributeError:
        pass
    
    all_isnan = np.any([~np.isfinite(bt), ~np.isfinite(wvd), ~np.isfinite(swd)], 0)
    
    bt.data[all_isnan] = np.nan
    wvd.data[all_isnan] = np.nan
    swd.data[all_isnan] = np.nan
    
    bt = fill_time_gap_nan(bt, time_gap)
    wvd = fill_time_gap_nan(wvd, time_gap)
    swd = fill_time_gap_nan(swd, time_gap)
    
    wvd.name = "WVD"
    wvd.attrs["standard_name"] = wvd.name
    wvd.attrs["long_name"] = "water vapour difference"
    wvd.attrs["units"] = "K"

    bt.name = "BT"
    bt.attrs["standard_name"] = bt.name
    bt.attrs["long_name"] = "brightness temperature"
    bt.attrs["units"] = "K"

    swd.name = "SWD"
    swd.attrs["standard_name"] = swd.name
    swd.attrs["long_name"] = "split window difference"
    swd.attrs["units"] = "K"
    
    return bt, wvd, swd

In [33]:
seviri_ds = seviri_dataloader(datetime(2016,6,1,13), datetime(2016,6,1,14), n_pad_files=3)

In [34]:
seviri_ds[0].t

<xarray.DataArray 't' (t: 10)>
array(['2016-06-01T12:15:00.000000000', '2016-06-01T12:30:00.000000000',
       '2016-06-01T12:45:00.000000000', '2016-06-01T13:00:00.000000000',
       '2016-06-01T13:15:00.000000000', '2016-06-01T13:30:00.000000000',
       '2016-06-01T13:45:00.000000000', '2016-06-01T14:00:00.000000000',
       '2016-06-01T14:15:00.000000000', '2016-06-01T14:30:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * t        (t) datetime64[ns] 2016-06-01T12:15:00 ... 2016-06-01T14:30:00

In [27]:
find_seviri_files(datetime(2016,6,1,13), datetime(2016,6,1,14))

['../data/SEVIRI_ORAC/H-000-MSG3__-MSG3________-_________-EPI______-201606011245-__.secondary.nc',
 '../data/SEVIRI_ORAC/H-000-MSG3__-MSG3________-_________-EPI______-201606011300-__.secondary.nc',
 '../data/SEVIRI_ORAC/H-000-MSG3__-MSG3________-_________-EPI______-201606011315-__.secondary.nc',
 '../data/SEVIRI_ORAC/H-000-MSG3__-MSG3________-_________-EPI______-201606011330-__.secondary.nc',
 '../data/SEVIRI_ORAC/H-000-MSG3__-MSG3________-_________-EPI______-201606011345-__.secondary.nc',
 '../data/SEVIRI_ORAC/H-000-MSG3__-MSG3________-_________-EPI______-201606011400-__.secondary.nc']

In [35]:
seviri_ds[0]

<xarray.DataArray 'BT' (t: 10, along_track: 1601, across_track: 2081)>
array([[[270.05   , 271.02   , 271.02   , ..., 260.57   , 260.75   ,
         261.47998],
        [270.53998, 271.02   , 270.21   , ..., 260.57   , 260.39   ,
         260.57   ],
        [270.38   , 270.21   , 269.72   , ..., 261.66   , 261.11   ,
         260.02   ],
        ...,
        [309.55   , 310.     , 306.13   , ..., 283.41998, 285.65997,
         288.     ],
        [308.88   , 305.08002, 303.19   , ..., 284.97   , 289.21   ,
         290.94   ],
        [306.36   , 301.88   , 304.49   , ..., 290.55   , 292.39   ,
         292.26   ]],

       [[269.22998, 269.56   , 269.89   , ..., 260.93   , 260.39   ,
         260.2    ],
        [270.38   , 271.02   , 270.53998, ..., 260.39   , 258.91   ,
         258.53998],
        [271.51   , 270.86   , 270.05   , ..., 260.75   , 258.34998,
         258.91   ],
...
        [302.36   , 299.83002, 294.96   , ..., 293.81   , 293.68   ,
         293.68   ],
        [300.31   , 294.96   , 283.7    , ..., 293.68   , 293.68   ,
         293.68   ],
        [290.40997, 283.7    , 275.9    , ..., 293.68   , 293.68   ,
         293.68   ]],

       [[270.38   , 270.38   , 270.05   , ..., 274.36   , 276.82   ,
         279.8    ],
        [269.39   , 269.56   , 269.72   , ..., 271.34998, 272.47   ,
         279.51   ],
        [269.39   , 269.56   , 269.39   , ..., 270.53998, 271.19   ,
         275.59   ],
        ...,
        [292.26   , 290.55   , 283.56   , ..., 293.68   , 293.81   ,
         293.81   ],
        [275.28998, 272.15   , 273.72998, ..., 293.68   , 293.81   ,
         293.81   ],
        [265.7    , 269.06   , 278.16998, ..., 293.68   , 293.81   ,
         293.81   ]]], dtype=float32)
Coordinates:
  * t        (t) datetime64[ns] 2016-06-01T12:15:00 ... 2016-06-01T14:30:00
Dimensions without coordinates: along_track, across_track
Attributes:
    long_name:      brightness temperature
    standard_name:  BT
    units:          K
    valid_min:      0
    valid_max:      32000

In [36]:
goes_ds

<xarray.Dataset>
Dimensions:                                 (band: 1, number_of_image_bounds: 2, number_of_time_bounds: 2, x: 2500, y: 1500)
Coordinates:
    t                                       datetime64[ns] 2018-06-19T18:03:4...
  * y                                       (y) float32 0.1282 ... 0.04427
  * x                                       (x) float32 -0.1013 ... 0.03861
    y_image                                 float32 0.08624
    x_image                                 float32 -0.03136
    band_wavelength_C01                     (band) float32 0.47
    band_wavelength_C02                     (band) float32 0.64
    band_wavelength_C03                     (band) float32 0.865
    band_wavelength_C04                     (band) float32 1.378
    band_wavelength_C05                     (band) float32 1.61
    band_wavelength_C06                     (band) float32 2.25
    band_wavelength_C07                     (band) float32 3.89
    band_wavelength_C08                     (band) float32 6.17
    band_wavelength_C09                     (band) float32 6.93
    band_wavelength_C10                     (band) float32 7.34
    band_wavelength_C11                     (band) float32 8.44
    band_wavelength_C12                     (band) float32 9.61
    band_wavelength_C13                     (band) float32 10.33
    band_wavelength_C14                     (band) float32 11.19
    band_wavelength_C15                     (band) float32 12.27
    band_wavelength_C16                     (band) float32 13.27
    band_id_C01                             (band) int8 1
    band_id_C02                             (band) int8 2
    band_id_C03                             (band) int8 3
    band_id_C04                             (band) int8 4
    band_id_C05                             (band) int8 5
    band_id_C06                             (band) int8 6
    band_id_C07                             (band) int8 7
    band_id_C08                             (band) int8 8
    band_id_C09                             (band) int8 9
    band_id_C10                             (band) int8 10
    band_id_C11                             (band) int8 11
    band_id_C12                             (band) int8 12
    band_id_C13                             (band) int8 13
    band_id_C14                             (band) int8 14
    band_id_C15                             (band) int8 15
    band_id_C16                             (band) int8 16
Dimensions without coordinates: band, number_of_image_bounds, number_of_time_bounds
Data variables:
    CMI_C01                                 (y, x) float32 ...
    DQF_C01                                 (y, x) float32 ...
    CMI_C02                                 (y, x) float32 ...
    DQF_C02                                 (y, x) float32 ...
    CMI_C03                                 (y, x) float32 ...
    DQF_C03                                 (y, x) float32 ...
    CMI_C04                                 (y, x) float32 ...
    DQF_C04                                 (y, x) float32 ...
    CMI_C05                                 (y, x) float32 ...
    DQF_C05                                 (y, x) float32 ...
    CMI_C06                                 (y, x) float32 ...
    DQF_C06                                 (y, x) float32 ...
    CMI_C07                                 (y, x) float32 ...
    DQF_C07                                 (y, x) float32 ...
    CMI_C08                                 (y, x) float32 ...
    DQF_C08                                 (y, x) float32 ...
    CMI_C09                                 (y, x) float32 ...
    DQF_C09                                 (y, x) float32 ...
    CMI_C10                                 (y, x) float32 ...
    DQF_C10                                 (y, x) float32 ...
    CMI_C11                                 (y, x) float32 ...
    DQF_C11                                 (y, x) float32 ...
    CMI_C12                              